In [493]:
import pandas as pd
import pymystem3 
import spacy
nlp = spacy.load("ru_core_news_lg")

### Data Load

In [494]:
df_manual = []
with open("../edz_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line, 1])
        
with open("../edz_neg_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line, 0])
df_manual = pd.DataFrame(df_manual, columns=["text", "label"])


In [495]:
df_manual

,text,label
0,завтра буду учить английский весь день,1
1,Завтра я пойду на работу,1
2,послезавтра мы планируем ехать на дачу,1
3,к вечеру я планирую закончить отчет,1
4,в воскресенье после обеда мы пойдем в ресторан,1
...,...,...
496,Мой отец увлекается историей автомобилей и кол...,0
497,Знакомый одноклассник работает в издательстве ...,0
498,Моя сестра изучает психологию и помогает людям...,0
499,Твой отец увлекается историей автомобилей и ко...,0


Разберем предложения с планами и выделим особенности, которые возможно понадобятся при обучении моделей

#### SpaCy

In [496]:
tags = []
doc = nlp(df_manual.loc[0, "text"])
for token in doc:
    tags.append([token.text, token.lemma_, token.ent_type_, token.pos_, token.tag_, token.dep_, token.head])
postok = {str(i): list(doc).index(i) for i in doc}
pd.DataFrame(tags, columns = ["Token", "lemma", "entity", "full", "entity", "dep type", "head"])

,Token,lemma,entity,full,entity,dep type,head
0,завтра,завтра,,ADV,ADV,advmod,учить
1,буду,буду,,AUX,AUX,aux,учить
2,учить,учить,,VERB,VERB,ROOT,учить
3,английский,английский,,ADJ,ADJ,obj,учить
4,весь,весь,,DET,DET,det,день
5,день,день,,NOUN,NOUN,obl,учить


#### MyStem

In [497]:
stem = pymystem3.Mystem()
stem.analyze(df_manual.loc[0, "text"])

[{'analysis': [{'lex': 'завтра', 'wt': 0.9964678515, 'gr': 'ADV='}],
  'text': 'завтра'},
 {'text': ' '},
 {'analysis': [{'lex': 'быть',
    'wt': 0.8530947255,
    'gr': 'V,нп=непрош,ед,изъяв,1-л'}],
  'text': 'буду'},
 {'text': ' '},
 {'analysis': [{'lex': 'учить', 'wt': 1, 'gr': 'V,несов,пе=инф'}],
  'text': 'учить'},
 {'text': ' '},
 {'analysis': [{'lex': 'английский',
    'wt': 1,
    'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}],
  'text': 'английский'},
 {'text': ' '},
 {'analysis': [{'lex': 'весь',
    'wt': 0.9978909737,
    'gr': 'APRO=(вин,ед,муж,неод|им,ед,муж)'}],
  'text': 'весь'},
 {'text': ' '},
 {'analysis': [{'lex': 'день',
    'wt': 0.999920149,
    'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'день'},
 {'text': '\n'}]

#### Natasha

In [499]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
text = df_manual.loc[23, "text"]
doc = Doc(text)

In [500]:
doc.segment(segmenter=segmenter)
doc.tag_morph(tagger=morph_tagger)
for token in doc.tokens:
    token.lemmatize(morph_vocab)

In [501]:
doc.sents[0].morph.print()

                   Я PRON|Case=Nom|Number=Sing|Person=1
               поеду VERB|Aspect=Perf|Mood=Ind|Number=Sing|Person=1|Tense=Fut|VerbForm=Fin|Voice=Act
                   в ADP
              отпуск NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                этим DET|Case=Ins|Gender=Neut|Number=Sing
               летом NOUN|Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing
                  На ADP
                Бали PROPN|Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing


In [502]:
doc.tag_ner(ner_tagger)

In [503]:
doc.ner

NERMarkup(
    text='Я поеду в отпуск этим летом На Бали',
    spans=[Span(
         start=31,
         stop=35,
         type='LOC'
     )]
)

У рута-глагола будет полезным выделить признак стоит ли глагол в Perfect времени

In [504]:
[doc.sents[0].morph.tokens[1].feats["Aspect"],
doc.sents[0].morph.tokens[1].feats["Tense"], 
doc.sents[0].morph.tokens[1].feats["VerbForm"]]

['Perf', 'Fut', 'Fin']

## ML модели

### BoW

In [505]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Выделяем леммы слов

In [506]:
def preprocess(text):
    lemmas = []
    doc = nlp(text)
    for tok in doc:
        lemmas.append(tok.lemma_)
    return " ".join(lemmas)

In [507]:
df_manual["lemmas"] = df_manual["text"].apply(lambda x: preprocess(x))
model_bow = CountVectorizer()
X = model_bow.fit_transform(df_manual.lemmas).toarray()
y = df_manual.label
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True)

Используем RandomForest для классификации

In [508]:
model_rfr = RandomForestClassifier()
model_rfr.fit(x_train, y_train)

RandomForestClassifier()

Оценивать модели будем путем деления выборки на обучающую и тестирующую часть. Будем использовать Accuracy, F1, площадь под ROC кривой

In [509]:
from sklearn.metrics import classification_report
y_pred = model_rfr.predict(x_test)
print("Random Forest Classifier")
print(classification_report(y_test, y_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_pred)}")

Random Forest Classifier
              precision    recall  f1-score   support

           0       0.97      0.69      0.80        48
           1       0.84      0.99      0.91        78

    accuracy                           0.87       126
   macro avg       0.90      0.84      0.86       126
weighted avg       0.89      0.87      0.87       126

Test ROC AUC: 0.8373397435897436


И протестируем более сложную модель - градиентный бустинг (sklearn)

In [510]:
from sklearn.ensemble import GradientBoostingClassifier
model_gb = GradientBoostingClassifier()
model_gb.fit(x_train, y_train)

GradientBoostingClassifier()

In [511]:
from sklearn.metrics import classification_report
y_pred = model_gb.predict(x_test)
print("Gradient Boosting")
print(classification_report(y_test, y_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_pred)}")

Gradient Boosting
              precision    recall  f1-score   support

           0       0.85      0.83      0.84        48
           1       0.90      0.91      0.90        78

    accuracy                           0.88       126
   macro avg       0.87      0.87      0.87       126
weighted avg       0.88      0.88      0.88       126

Test ROC AUC: 0.8717948717948717


### Word2Vec

In [513]:
x_train, x_test, y_train, y_test = train_test_split(df_manual["lemmas"], df_manual["label"])

Будем использовать токенизатор keras

In [514]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf

SENTENCE_LENGTH = 20
NUM = 200

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

# Отображаем каждый текст в массив идентификаторов токенов
x_train_seq = get_sequences(tokenizer, x_train)
x_test_seq = get_sequences(tokenizer, x_test)

Сделаем нейросеть из 15 ядер свертки с различными конфигурациями с GlobalMaxPooling, а из выводы подадим полносвязному слою.
Ембеддинг возьмем стандартный из библиотеки keras

In [515]:
from keras.layers import Input, Embedding, Dropout, Conv1D, Activation, Dense, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

input_layer = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
x = Embedding(NUM, 50, input_length=SENTENCE_LENGTH,trainable=True)(input_layer)
branches =  []

for size in range(2, 5):
    for i in range(5):
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='relu')(x)
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)

x = concatenate(branches, axis=1)
x = Dropout(0.25)(x)
x = Dense(16, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model_w2v = Model(inputs=[input_layer], outputs=[output])

model_w2v.compile(loss=BinaryCrossentropy(), optimizer=Adam(), metrics=['accuracy'])
model_w2v.summary()      

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_33 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 embedding_32 (Embedding)       (None, 20, 50)       10000       ['input_33[0][0]']               
                                                                                                  
 conv1d_801 (Conv1D)            (None, 19, 1)        101         ['embedding_32[0][0]']           
                                                                                                  
 conv1d_802 (Conv1D)            (None, 19, 1)        101         ['embedding_32[0][0]']           
                                                                                           

In [516]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='auto')

history_w2v = model_w2v.fit(x_train_seq, y_train, validation_split=0.2, epochs=60,callbacks=early_stop)

Epoch 1/60
10/10 [==============================] - 8s 171ms/step - loss: 0.6881 - accuracy: 0.5367 - val_loss: 0.6731 - val_accuracy: 0.6400
Epoch 2/60
10/10 [==============================] - 1s 60ms/step - loss: 0.6674 - accuracy: 0.6267 - val_loss: 0.6534 - val_accuracy: 0.6400
Epoch 3/60
10/10 [==============================] - 1s 58ms/step - loss: 0.6479 - accuracy: 0.6267 - val_loss: 0.6310 - val_accuracy: 0.6400
Epoch 4/60
10/10 [==============================] - 1s 61ms/step - loss: 0.6158 - accuracy: 0.6400 - val_loss: 0.6033 - val_accuracy: 0.6400
Epoch 5/60
10/10 [==============================] - 1s 62ms/step - loss: 0.5863 - accuracy: 0.6700 - val_loss: 0.5680 - val_accuracy: 0.6800
Epoch 6/60
10/10 [==============================] - 1s 57ms/step - loss: 0.5406 - accuracy: 0.7400 - val_loss: 0.5241 - val_accuracy: 0.7600
Epoch 7/60
10/10 [==============================] - 1s 61ms/step - loss: 0.4904 - accuracy: 0.8067 - val_loss: 0.4769 - val_accuracy: 0.8667
Epoch 8/60
1

In [517]:
y_pred = np.array(model_w2v.predict(x_test_seq) > 0.5)
print("CNN - 15 conv layers")
print(classification_report(y_test, y_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_pred)}")

4/4 [==============================] - 1s 7ms/step
CNN - 15 conv layers
              precision    recall  f1-score   support

           0       0.91      0.86      0.89        59
           1       0.89      0.93      0.91        67

    accuracy                           0.90       126
   macro avg       0.90      0.89      0.90       126
weighted avg       0.90      0.90      0.90       126

Test ROC AUC: 0.8948899569946877


Попробуем увеличить количество ядер свертки до 48

In [518]:
from keras.layers import Input, Embedding, Dropout, Conv1D, Activation, Dense, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

input_layer = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
x = Embedding(NUM, 50, input_length=SENTENCE_LENGTH,trainable=True)(input_layer)
branches =  []

for size in range(2, 5):
    for i in range(16):
        branch = Conv1D(filters=5, kernel_size=size, padding='valid', activation='relu')(x)
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)

x = concatenate(branches, axis=1)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu')(x)
x = Dense(10, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model_w2v = Model(inputs=[input_layer], outputs=[output])

model_w2v.compile(loss=BinaryCrossentropy(), optimizer=Adam(), metrics=['accuracy'])
model_w2v.summary()      

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_34 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 embedding_33 (Embedding)       (None, 20, 50)       10000       ['input_34[0][0]']               
                                                                                                  
 conv1d_816 (Conv1D)            (None, 19, 5)        505         ['embedding_33[0][0]']           
                                                                                                  
 conv1d_817 (Conv1D)            (None, 19, 5)        505         ['embedding_33[0][0]']           
                                                                                           

In [519]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='auto')

history_w2v = model_w2v.fit(x_train_seq, y_train, validation_split=0.2, epochs=60,callbacks=early_stop)

Epoch 1/60
10/10 [==============================] - 24s 727ms/step - loss: 0.6784 - accuracy: 0.6200 - val_loss: 0.6530 - val_accuracy: 0.6400
Epoch 2/60
10/10 [==============================] - 4s 425ms/step - loss: 0.6225 - accuracy: 0.6700 - val_loss: 0.5925 - val_accuracy: 0.7333
Epoch 3/60
10/10 [==============================] - 4s 434ms/step - loss: 0.5272 - accuracy: 0.8500 - val_loss: 0.4998 - val_accuracy: 0.8400
Epoch 4/60
10/10 [==============================] - 4s 437ms/step - loss: 0.3869 - accuracy: 0.9267 - val_loss: 0.3700 - val_accuracy: 0.8667
Epoch 5/60
10/10 [==============================] - 4s 431ms/step - loss: 0.2263 - accuracy: 0.9533 - val_loss: 0.2956 - val_accuracy: 0.8667
Epoch 6/60
10/10 [==============================] - 4s 434ms/step - loss: 0.1373 - accuracy: 0.9633 - val_loss: 0.2923 - val_accuracy: 0.8933
Epoch 7/60
10/10 [==============================] - 4s 442ms/step - loss: 0.0732 - accuracy: 0.9833 - val_loss: 0.3073 - val_accuracy: 0.9067
Epoch

In [520]:
y_pred = np.array(model_w2v.predict(x_test_seq) > 0.5)
print("CNN - 48 conv layers")
print(classification_report(y_test, y_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_pred)}")

4/4 [==============================] - 1s 39ms/step
CNN - 48 conv layers
              precision    recall  f1-score   support

           0       0.88      0.86      0.87        59
           1       0.88      0.90      0.89        67

    accuracy                           0.88       126
   macro avg       0.88      0.88      0.88       126
weighted avg       0.88      0.88      0.88       126

Test ROC AUC: 0.8799645838603593


На наших данных увеличение глубины модели не дало улучшение результата

### Тестируем модель  CNN

In [547]:
res = model_w2v.predict(get_sequences(tokenizer, [preprocess("Мы в следующем месяце хотим поступить на бюджет")]))
res

1/1 [==============================] - 0s 23ms/step


array([[0.9967316]], dtype=float32)

In [552]:
res = model_w2v.predict(get_sequences(tokenizer, [preprocess("Я не собираюсь есть кашу с изюмом завтра")]))
res

1/1 [==============================] - 0s 29ms/step


array([[0.38332325]], dtype=float32)

In [553]:
res = model_w2v.predict(get_sequences(tokenizer, [preprocess("Я собираюсь есть кашу с изюмом завтра")]))
res

1/1 [==============================] - 0s 32ms/step


array([[0.990744]], dtype=float32)

In [554]:
res = model_w2v.predict(get_sequences(tokenizer, [preprocess("Мне не нравится каша с изюмом")]))
res

1/1 [==============================] - 0s 21ms/step


array([[0.00186641]], dtype=float32)